In [1]:
import os
import pandas as pd
import numpy as np
import re
import string

In [3]:
def percent_by_polling_district(folderName, riding, year):
    print "Year: " + str(int(year)) + ", Riding: " + str(int(riding))
    fileName = "pollresults_resultatsbureau" + str(int(riding)) + ".csv"
    filePath = os.path.join(folderName, fileName)
    # Load the data
    if (year == 2015):
        # The encoding for 2015 is different from previous years
        pollData = pd.read_csv(filePath, encoding="utf-8")
    else:
        pollData = pd.read_csv(filePath)
    
    # Get column names, and remove French portions
    colNames = list(pollData.columns.values)
    colNames = [x.split('/')[0] for x in colNames]
    # Remove unicode apostrophes
    colNames = [x.replace(u"\u2019", "'") for x in colNames]
    pollData.columns = colNames
    
    # Drop unnecessary columns
    listColDrop = ['Electoral District Name_English',
                   'Electoral District Name_French',
                   'No Poll Held Indicator', 
                   'Merge With',
                   'Rejected Ballots for Polling Station',
                   'Political Affiliation Name_French',
                   "Candidate's First Name",
                   "Candidate's Family Name",
                   "Candidate's Middle Name"]
    pollData = pollData.drop(listColDrop, axis=1)
    
    # Strip the polling ID column of whitespace.
    polCol = 'Polling Station Number'
    s = lambda x: str(x).strip(" ")
    pollData[polCol] = pollData[polCol].map(s)
    
    # Keep the total for the polls because we later remove smaller parties
    pollVoteTotals = pollData.groupby('Polling Station Number')['Candidate Poll Votes Count'].sum()
    pollVoteTotals = pd.DataFrame(pollVoteTotals)
    pollVoteTotals.reset_index(level=0, inplace=True)
    
    # Remove parties that aren't one of the main ones
    # parties = "Liberal|Conservative|NDP|N.D.P.|Green|Bloc|Wild|Marxist|Libertarian"
    # pollData = pollData[pollData['Political Affiliation Name_English'].str.contains(parties)]
    
    # Create a pivot table of the data by polling district/candidate name
    pollData = pollData.pivot(
        index='Polling Station Number',
        columns='Political Affiliation Name_English',
        values='Candidate Poll Votes Count')
    # Turn the index back into a column
    pollData.reset_index(level=0, inplace=True)
    
    # Strip the letters off polling stations since the geospatial data
    # does not include these letters and they are typically used for
    # mobile polls
    stripCharacters = "ABCDEFG"
    s = lambda x: str(x).strip(stripCharacters)
    statCol = 'Polling Station Number'

    pollData[statCol] = pollData[statCol].map(s)
    pollVoteTotals[statCol] = pollVoteTotals[statCol].map(s)
    
    # Merge polling stations since some were split in two parts
    pollData = pollData.groupby(statCol).sum()
    pollVoteTotals = pollVoteTotals.groupby(statCol).sum()
    pollsWithTotals = pd.concat([pollData, pollVoteTotals], axis=1)
    pollsWithTotals.rename(columns={'Candidate Poll Votes Count': 'Vote Totals'}, inplace=True)

    # Calculate the percent for each
    # Grab the data we want converted to a percent
    numColsPollData = len(pollsWithTotals.columns)
    pollDataPercent = pollsWithTotals.iloc[:, range(0, numColsPollData-1)].copy()

    # Divide it by the total votes for each polling district
    pollDataPercent = pollDataPercent.div(pollsWithTotals['Vote Totals'], axis=0)
    
    # Calculate the percent that every other party got
    # pollDataPercent['Other'] = 1 - pollDataPercent.sum(axis=1)
    # For rows with no votes, the other column becomes 100%
    # pollDataPercent.loc[pollDataPercent['Other'] == 1, 'Other'] = None
    
    # Round everything off
    pollDataPercent = np.round(pollDataPercent*100, decimals=2)

    # Rename columns
    colNames = list(pollDataPercent.columns.values)
    colNames = [x + " Percent" for x in colNames]
    pollDataPercent.columns = colNames

    # Merge it with the original data set
    pollData = pd.concat([pollData, pollDataPercent], axis=1)
    
    # Add back column for electoral district
    pollData['District'] = riding
    
    pollData.reset_index(level=0, inplace=True)
    return pollData

In [4]:
# Create an array for all the years we're interested in
years = [2008, 2011, 2015]

In [ ]:
for year in years:
    # Get list of riding numbers
    folderName = "election_results/pollresults_resultatsbureau_canada" + str(year)
    
    ### USED IF PERFORMING ANALYSIS FOR LARGE AREA
    # The 2006 data is missing a necessary column to automate this, so
    # we need to treat it differently
#     if year != 2006:
#         # This file contains the provinces and their electoral districts
#         ridingFile = os.path.join(folderName, "RidingListClean.csv")
#         ridingList = pd.read_csv(ridingFile)
#         # We only need the first three columns
#         ridingList = ridingList.iloc[:, [0, 2]]
#         ridingList = ridingList[ridingList["Province2"] == "New Brunswick"]
#         ridings = ridingList.ix[:, 1]
#         # Cleans up some floating point numbers
#         ridings = tuple(int(riding) for riding in ridings)
#     else:
#         ridings = [13001, 13002, 13003, 13004, 13005, 13006, 13007, 13008, 13009, 13010]
    if ((year == 2008) or (year == 2011)):
        ridings = [35038, 35037, 35039]
    elif (year == 2015):
        ridings = [35045, 35046, 35047, 35112]
    
    # Is this the first time looping for this year?
    first = True
    for riding in ridings:
        pollData = percent_by_polling_district(folderName, riding, year)
        if first == True:
            combined = pollData
        else:
            combined = pd.concat([combined, pollData], axis=0, ignore_index=True)
        first = False

    # Reorder columns before writing
    cols = list(combined)
    cols.insert(0, cols.pop(cols.index('Polling Station Number')))
    cols.insert(0, cols.pop(cols.index('District')))
    combined = combined.ix[:, cols]
    
    # Strip characters that make ArcGIS angry
    colNames = list(combined.columns.values)
    
    # Newer Elections Canada are in UTF-8 and the unicode encoding makes some things very difficult in Python 2
    colNames = [unicode(s).translate({ord(c): None for c in ' -./'}) for s in colNames]
    
    # identity = string.maketrans("", "")
    # colNames = [s.translate(identity, " -./") for s in colNames]
    # colNames = tuple(col.translate(None, ' -.') for col in colNames)
    combined.columns = colNames
    
    combined['unique_poll_id'] = combined['District'].map(str) + combined['PollingStationNumber'].map(str)
    combined.fillna(0, inplace=True)
    
    fileName = str(int(year)) + "Combined.xls"
    filePath = os.path.join(".", "election_results", "output", fileName)
    combined.to_excel(filePath, index=False, encoding='utf-8')